In [1]:
import torch
import torchvision
from torchvision import transforms

import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import CSVLogger

from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, ModelSummary
from pytorch_lightning.callbacks.progress import TQDMProgressBar


from torch.nn import functional as F
from torch.utils.data import DataLoader

#Metricas
import torchmetrics
from torchmetrics.classification import MulticlassAccuracy


from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


In [7]:
##########################
### Configurações
##########################

MODEL_NAME = "ResNet50"

# Hyperparameters
LEARNING_RATE = 1e-3
BATCH_SIZE = 8 if torch.cuda.is_available() else 64
NUM_WORKERS = 0 # Zero for Windows
NUM_EPOCHS = 3

# Architecture
NUM_CLASSES = 7

# Other
DEVICE = "cuda:0" # ou "CPU"

width_pic = 224
height_pic = 224


In [8]:
print(BATCH_SIZE)
print(DEVICE)
print(LEARNING_RATE)
#print(f"There are {len(train_dataset)} train images and {len(val_dataset)} val images")

8
cuda:0
0.001


In [9]:
PATH_DS_PROCESSED_TRAINING = Path("./IMG224_ALL/PROCESSED-TRAIN")
PATH_DS_PROCESSED_VALIDATION = Path("./IMG224_ALL/PROCESSED-VALID")
PATH_DS_PROCESSED_TEST = Path("./IMG224_ALL/PROCESSED-TEST")

pathname = "./WEIGHTS/" + MODEL_NAME + "/"
PATH_DS_WEIGHTS = Path(pathname)

In [10]:
def load_file(path):
    return np.load(path).astype(np.float32)

In [11]:
# Valores correspondente ao ds completo (todas as classificações)
# a normalização das imagens depende desses valores e devem ser levadas em consideração o dataset que está sendo analisado
mean = torch.tensor([118.9568,  76.7986,  43.6991]) 
stq = torch.tensor([71.9256, 50.9931, 35.3619])     

train_transforms = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.RandomRotation(degrees=(-45, 45)),
                                    transforms.Normalize(mean,stq),
])

val_transforms = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean,stq),
])


In [12]:
# Create the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath=PATH_DS_WEIGHTS,
    monitor='val_epoch_acc',
    save_top_k=3,
    mode='max')

In [13]:
#Datasets
TRAIN_DATASET = torchvision.datasets.DatasetFolder(
    PATH_DS_PROCESSED_TRAINING,
    loader=load_file, extensions="npy", transform=train_transforms)

VAL_DATASET = torchvision.datasets.DatasetFolder(
    PATH_DS_PROCESSED_VALIDATION,
    loader=load_file, extensions="npy", transform=val_transforms)

TEST_DATASET = torchvision.datasets.DatasetFolder(
    PATH_DS_PROCESSED_TEST,
    loader=load_file, extensions="npy", transform=val_transforms)

In [14]:
TRAIN_LOADER = torch.utils.data.DataLoader(TRAIN_DATASET, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
VAL_LOADER = torch.utils.data.DataLoader(VAL_DATASET, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)
TEST_LOADER = torch.utils.data.DataLoader(TEST_DATASET, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

In [15]:
class LTResNet50Module(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.learning_rate = LEARNING_RATE      
        
        self.model = torchvision.models.resnet50(pretrained=True)
        
        fc_in_features = self.model.fc.in_features
        self.model.fc = torch.nn.Linear(in_features=fc_in_features, out_features=NUM_CLASSES, bias=True)
        
        self.loss_fn = torch.nn.CrossEntropyLoss()

        self.metric_train_accuracy = torchmetrics.classification.MulticlassAccuracy(num_classes=NUM_CLASSES)
        self.metric_val_accuracy = torchmetrics.classification.MulticlassAccuracy(num_classes=NUM_CLASSES)   
        self.metric_test_accuracy = torchmetrics.classification.MulticlassAccuracy(num_classes=NUM_CLASSES)
        
        self.metric_train_f1score = torchmetrics.classification.MulticlassF1Score(num_classes=NUM_CLASSES)
        self.metric_val_f1score = torchmetrics.classification.MulticlassF1Score(num_classes=NUM_CLASSES)    
        self.metric_test_f1score = torchmetrics.classification.MulticlassF1Score(num_classes=NUM_CLASSES)

    def forward(self, x):
        x = self.model(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        #optimizer = torch.optim.SGD(self.parameters(), **self.hparams.optimizer_hparams)
        # We will reduce the learning rate by 0.1 after 100 and 150 epochs
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1,3,5,7,8,10,15,20], gamma=0.5)

        return [optimizer], [scheduler]
        
    def training_step(self, batch, batch_idx):
        x, labels = batch
        preds = self.model(x)

        loss = self.loss_fn(preds, labels)
        
        pred_labels = torch.argmax(preds, dim=1)
        
        self.metric_train_accuracy.update(pred_labels, labels)
        self.metric_train_f1score.update(pred_labels, labels)
        
        self.log("train_st_loss", loss)
        
        return loss
    
    def training_epoch_end(self, outs):
        self.log("train_epoch_acc", self.metric_train_accuracy.compute())
        self.log("train_epoch_f1score", self.metric_train_f1score.compute())
        self.metric_train_accuracy.reset()
        self.metric_train_f1score.reset()

    def validation_step(self, batch, batch_idx):
        x, labels = batch
        preds = self.model(x)
        loss = self.loss_fn(preds, labels)
        pred_labels = torch.argmax(preds, dim=1)
        
        self.metric_val_accuracy.update(pred_labels, labels)
        self.metric_val_f1score.update(pred_labels, labels)
        
        self.log("val_st_loss", loss)
        
    def validation_epoch_end(self, outs):
        self.log("val_epoch_acc", self.metric_val_accuracy.compute(), prog_bar=True)
        self.log("val_epoch_f1score", self.metric_val_f1score.compute())
        self.metric_val_accuracy.reset()
        self.metric_val_f1score.reset()

    def test_step(self, batch, batch_idx):
        x, labels = batch
        preds = self.model(x).argmax(dim=-1)
        acc = (labels == preds).float().mean()
        self.log("test_acc", acc)

        

In [16]:
model = LTResNet50Module()

trainer = pl.Trainer(accelerator='gpu', devices=1, 
                                   logger=TensorBoardLogger("./lightning_logs", name=MODEL_NAME),
                                   log_every_n_steps=50,

                                callbacks=[checkpoint_callback, 
                                           #ModelSummary(max_depth=-1), 
                                           #LearningRateMonitor("epoch")
                                          ],
                                max_epochs=NUM_EPOCHS
                    )

trainer.fit(model, TRAIN_LOADER, VAL_LOADER)


C:\Users\AsusTon\anaconda3\envs\tcc_evn\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\AsusTon\anaconda3\envs\tcc_evn\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./lightning_logs\ResNet50
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type               | Params
-----------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\AsusTon\anaconda3\envs\tcc_evn\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\AsusTon\anaconda3\envs\tcc_evn\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.
